### Instal Ollama :

curl -fsSL https://ollama.com/install.sh | sh

ollama pull llama3-groq-tool-use:8b

ollama pull mistral:7b 

ollama pull nomic-embed-text

## Initialize The Graph :

In [ ]:
from typing import List,Dict
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START,END

In [ ]:
class State(TypedDict):
    Tool_Choise:str
    Grade:str
    Question:str
    Generation:str
    Context : str


graph_builder = StateGraph(State)

## Building The Tools and LLMs:

- Tools are already built in My_Tools.py

In [ ]:
from My_Tools import RAG_System,OpenWeather,Tavily,RED,YELLOW,BLUE,GREEN,RESET

- Let's build our LLMs

### Chooser:

In [ ]:
# Chooser LLM:

from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI


# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["RAG_System", "Tavily","OpenWeather"] = Field(...,description="Given a user question choose to route it to web search call Tavily or a database call RAG_System or tool for Weather Informations call OpenWeather",)


# LLM with Structered Output
llm = ChatOpenAI(api_key="ollama",model="llama3-groq-tool-use:8b",base_url="http://localhost:11434/v1")
structured_llm_router = llm.with_structured_output(RouteQuery)

# Prompt
system = """You are an expert at routing a user question to a RAG_System , Tavily or OpenWeather.\n\n
Tavily is a web search engine like Google.for questions related to News people and general questions\n
OpenWeather is an API that give the Weathr informations for a Location like City.\n
The RAG_System contains documents related to agents, prompt engineering, and adversarial attacks.\n
Use the RAG_System for questions on these topics. and Use OpenWeather for questions related to weather.  Otherwise, use Tavily web serach engine."""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

Chooser = route_prompt | structured_llm_router

In [ ]:
print(Chooser.invoke({"question": "It is raining Now in Meknes ?"}))
print(Chooser.invoke({"question": "What are the types of agent memory?"}))

In [ ]:
print(Chooser.invoke({"question": "What happens to Trump last days ?"}))

### Context_Grader :

In [ ]:
# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance of Informations."""

    binary_score: Literal["yes","no"] = Field(...,description="A binary_score to evaluate External Informations are relevant to the question, 'yes' or 'no'")


# LLM with function call
llm = ChatOpenAI(api_key="ollama",model="llama3-groq-tool-use:8b",base_url="http://localhost:11434/v1")
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """You are an expert in evaluation of the relevance of an Informations to a User question
if the information are relevent to the question respond by 'yes' else respond by 'no' """

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Informations: \n\n {Context} \n\n User question: {question}"),
    ]
)

Context_Grader = grade_prompt | structured_llm_grader

- Test

In [ ]:
question = "agent memory ?"
Context = RAG_System.invoke(question)
response = Context_Grader.invoke({"Context": Context,"question": question})
print(response)

### Rewriter
- In the Chain we use StrOutputParser() function to convert the response of the LLM to a string.

In [ ]:
### Question Re-writer
from langchain_core.output_parsers import StrOutputParser
# LLM
llm = ChatOpenAI(api_key="ollama",model="mistral:7b",base_url="http://localhost:11434/v1")


# Prompt
system = """You are an expert question re-writer that converts an input question to a better version that is optimized \n 
     for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning. Give Just the New question
     No additional details """
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

Rewriter = re_write_prompt | llm | StrOutputParser()

- Test

In [ ]:
question = "What is prompt engenering ?"
response = Rewriter.invoke({"question": question}).strip().strip('"')
response

### Reporter

In [ ]:
### Generate

from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = hub.pull("rlm/rag-prompt")
# LLM
llm = ChatOpenAI(api_key="ollama",model="mistral:7b",base_url="http://localhost:11434/v1",temperature=0)
# Chain
Reporter = prompt | llm | StrOutputParser()


- Test

In [ ]:
question ="adv attacks and LLMs?"
Context = RAG_System.invoke({"query":question})
print(Context)

In [ ]:
question = "agent memory ?"
Context = RAG_System.invoke(question)
response = Reporter.invoke({"context": Context, "question": question})
print(response)

## Nodes Functions :

In [37]:
Num_trails = 6

### LLMs functions

In [ ]:
def Chooser_function(state:State):
    # Sometimes LLM can return None, but if we rerun it returns the good response
    # So we well create a loop with number trials
    for i in range(Num_trails):
        print(f"{GREEN}Chooser Trial :{i}{RESET}")
        response = Chooser.invoke({"question": state["Question"]})
        if response:
             return {"Tool_Choise":response.datasource}
    raise ValueError("Chooser can't choise")

def Grader_function(state:State):
    # Sometimes LLM can return None, but if we rerun it returns the good response
    # So we well create a loop with a number trials
    for i in range(Num_trails):
        print(f"{GREEN}Grader Trial :{i}{RESET}")
        response = Context_Grader.invoke({"Context": state["Context"],"question": state["Question"]})
        if response:
             return {"Grade":response.binary_score}
    raise ValueError("Grader can't evaluate Context")

def Rewriter_function(state:State):

    print(f"{GREEN}Rewriter{RESET}")
    response = Rewriter.invoke({"question": state["Question"]}).strip().strip('"')
    # Update the State
    return {"Question" : response}

def Reporter_function(state:State):

    print(f"{GREEN}Reporter{RESET}")
    response = Reporter.invoke({"context": state["Context"], "question": state["Question"]})
    # Update the State
    return {"Generation" : response}
    

### Tools functions:

In [ ]:
# The 3 tools have the same execution process so we will define one function for them
def Tool_function(state:State):
    Tool_Name = state["Tool_Choise"]
    query = state["Question"]
    print(f"{YELLOW}{Tool_Name} is executing{RESET}")
    
    if Tool_Name == "RAG_System":
        context = RAG_System.invoke({"query":query})
        return {"Context":context}

    if Tool_Name == "Tavily":
        docs = Tavily.invoke({"query": query})
        context = "\n\n---\n\n".join([d["content"] for d in docs])
        return {"Context":context}

    if Tool_Name == "OpenWeather":
        pass

### Conditional edges function:

In [ ]:
def Tool_Choise(state:State) -> str:
    return state["Tool_Choise"] # Name of the Tool Node

def Grade(state:State) -> str:
    if state["Grade"] == "yes":
        return "Reporter"
    return "Rewriter"

## Design the ARAG_Graph :

In [ ]:
graph_builder.add_node("Chooser",Chooser_function)
graph_builder.add_node("Grader",Grader_function)
graph_builder.add_node("Reporter",Reporter_function)
graph_builder.add_node("Rewriter",Rewriter_function)
graph_builder.add_node("Tavily",Tool_function)
graph_builder.add_node("RAG_System",Tool_function)
graph_builder.add_node("OpenWeather",Tool_function)

In [ ]:
graph_builder.add_edge(START,"Chooser")
graph_builder.add_conditional_edges("Chooser",Tool_Choise,{"Tavily":"Tavily","RAG_System":"RAG_System","OpenWeather":"OpenWeather"})
graph_builder.add_edge("Tavily","Grader")
graph_builder.add_edge("RAG_System","Grader")
graph_builder.add_edge("OpenWeather","Grader")
graph_builder.add_conditional_edges("Grader",Grade,{"Reporter":"Reporter","Rewriter":"Rewriter"})
graph_builder.add_edge("Rewriter","Chooser")
graph_builder.add_edge("Reporter",END)

In [ ]:
ARAG_Graph = graph_builder.compile()

## Visualize The Graph

In [ ]:
from IPython.display import Image, display

try:
    display(Image(ARAG_Graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

## Interact With the Graph

In [47]:
# Run
inputs = {
    "Question": "What Prompt Enegenering?"
}
for output in ARAG_Graph.stream(inputs):
    for key, value in output.items():
        # Node
        print(f"Node :{key}:")
        print(f"State :{value}")
    print("\n")

Chooser Trial :0


Chooser Trial :1
Chooser Trial :2
Node :Chooser:
State :{'Tool_Choise': 'RAG_System'}


RAG_System is executing


OllamaEmbeddings: 100%|██████████| 1/1 [00:01<00:00,  1.11s/it]


Node :RAG_System:
State :{'Context': 'Prompt Engineering | Lil\'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil\'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\n\n\n\nFAQ\n\n\n\n\nemojisearch.app\n\n\n\n\n\n\n\n\n\n      Prompt Engineering\n    \nDate: March 15, 2023  |  Estimated Reading Time: 21 min  |  Author: Lilian Weng\n\n\n \n\n\nTable of Contents\n\n\n\nBasic Prompting\n\nZero-Shot\n\nFew-shot\n\nTips for Example Selection\n\nTips for Example Ordering\n\n\n\nInstruction Prompting\n\nSelf-Consistency Sampling\n\nChain-of-Thought (CoT)\n\nTypes of CoT prompts\n\nTips and Extensions\n\n\nAutomatic Prompt Design\n\nAugmented Language Models\n\nRetrieval\n\nProgramming Language\n\nExternal APIs\n\n\nCitation\n\nUseful Resources\n\nReferences\n\n---\n\nFig. 8. Illustration of where adversarial triggers are introduced. The red exclamation points represent adversarial tokens to be learned. (

In [48]:
# Final generation
print(value["Generation"])

 Prompt Engineering refers to the process of designing and optimizing prompts (sequences of tokens) to increase the likelihood of getting a desired output from a language model given an input. This can be done by treating prompts as trainable parameters and optimizing them directly on the embedding space via gradient descent, or by using methods like Automatic Prompt Engineer (APE) to search over a pool of model-generated instruction candidates. The goal is to find an instruction that maximizes the expected score function for a given dataset of input-output pairs.
